In [ ]:
#Keras Function API
import keras
from keras.layers import Input, Dense, Flatten, Activation, Conv2D, BatchNormalization, Dropout, MaxPooling2D, AveragePooling2D
from keras.regularizers import l2
from keras.models import Model
from keras.datasets import cifar10
import tensorflow as tf
import numpy as np

batch_size = 32
num_classes = 10
num_epochs = 20

class CIFAR10_Data:
    def __init__(self):
        (x_train, y_train), (x_test, y_test) = cifar10.load_data()
        self.img_shape = (32,32,3)
        self.x_train = x_train.reshape(x_train.shape[0], self.img_shape[0], self.img_shape[1], self.img_shape[2])/255.0
        self.y_train = keras.utils.to_categorical(y_train, num_classes)
        self.x_test = x_test.reshape(x_test.shape[0], self.img_shape[0], self.img_shape[1], self.img_shape[2])/255.0
        self.y_test = keras.utils.to_categorical(y_test, num_classes)
    def train(self):
        indices = np.arange(len(y_train))
        random_indices = np.random.choice(indices)
        return self.x_train[random_indices], self.y_train[random_indices]
    def test(self):
        return self.x_test, self.y_test
    def input_shape(self):
        return self.img_shape

data = CIFAR10_Data()
def ComputationalGraph2(inputs):
    x = Conv2D(64, (3,3), kernel_initializer = 'he_normal', activation='relu')(inputs)
    x = MaxPooling2D((2,2))(x)
    x = Conv2D(128, (3,3), kernel_initializer = 'he_normal')(x)
    x = MaxPooling2D((2,2))(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Dropout(0.4)(x)

    x = Inception(x)
    x = Inception(x)
    x = MaxPooling2D((2,2))(x)
    x = BatchNormalization()(x)
    x = Dropout(0.2)(x)

    x = Inception(x)
    x = Inception(x)
    x = MaxPooling2D((2,2))(x)
    x = BatchNormalization()(x)
    x = Dropout(0.2)(x)

    x = Flatten()(x)
    x = Dense(128, kernel_regularizer = l2(0.01), activation = 'relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.2)(x)
    predictions = Dense(10, activation = 'softmax')(x)
    return predictions  

def Inception(x):
    conv_1 = Conv2D(64, (1, 1), padding='same', activation='relu')(x)
    conv_1 = Conv2D(64, (3, 3), padding='same', activation='relu')(conv_1)

    conv_2 = Conv2D(64, (1, 1), padding='same', activation='relu')(x)
    conv_2 = Conv2D(64, (5, 5), padding='same', activation='relu')(conv_2)

    conv_3 = MaxPooling2D((3, 3), strides=(1, 1), padding='same')(x)
    conv_3 = Conv2D(64, (1, 1), padding='same', activation='relu')(conv_3)
    x = keras.layers.concatenate([conv_1, conv_2, conv_3], axis=1)
    return x  
  
inputs = Input(shape = data.input_shape())
model = Model(inputs = inputs, outputs = ComputationalGraph2(inputs)) 
model.compile(optimizer='Adam',
    loss = 'categorical_crossentropy',
    metrics = ['accuracy'])

model.fit(data.x_train, data.y_train, verbose=True, batch_size = batch_size, epochs = 20, validation_split = 0.15)

score = model.evaluate(data.x_test, data.y_test, verbose=True)
print(score[0])
print(score[1])

Using TensorFlow backend.


Train on 42500 samples, validate on 7500 samples
Epoch 1/20
 5088/42500 [==>...........................] - ETA: 11:40 - loss: 3.5239 - acc: 0.2722